In [49]:
using StochasticPrograms
using LinearAlgebra
using Statistics
using HiGHS
using SparseArrays
using GLPK
using JuMP

solver = GLPK.Optimizer

GLPK.Optimizer

In [51]:
function firststage()
    m = Model(solver)

    @variable(m, 0<=x<=10)
    @variable(m, θ[i in 1:3])
    @objective(m, Min,
        0*x
    )
    
    return m, x, θ
end

firststage (generic function with 1 method)

In [53]:
function master_objective(m::Model, x, θ)
    @objective(m, Min,
        0*x+sum(θ[i] for i in 1:3)
    )
    return m
end

master_objective (generic function with 1 method)

In [55]:
function secondstageCore(x, demand)
    m = Model()

    @variable(m, y[i in 1:2] >= 0)
    
    T = [1]
    h = [demand]


    recourseConstraints = []

    return m, y, recourseConstraints, h, T
end 

secondstageCore (generic function with 1 method)

In [57]:
function secondstage(x, demand)
    m, y, recourseConstraints, h, T = secondstageCore(x, demand)

    push!(recourseConstraints, @constraint(m, y[1]-y[2] == demand - x))


    @objective(m, Min, y[1] + y[2])

    set_optimizer(m, solver)
    optimize!(m)
    
    return m, recourseConstraints, h, T
end

secondstage (generic function with 1 method)

In [99]:
function stop(expected_Q, θ, k, tol = 1e-10)
    nmax = 200  # circuit breaker
    for i in 1:3      
        if ((θ[i] > expected_Q) || isapprox(1e-20, θ[i]-expected_Q; atol=eps(Float64), rtol=0)|| (k == nmax))
            return true
        end
    end
    return false
end

stop (generic function with 2 methods)

In [101]:
function lshaped_multicut(scenarios, prob)
    nScenarios = 3
    
    k = 0       # iteration index
    nfcuts = 0  # number of feasibility cuts
    nocuts = 0  # number of optimality cuts
    
    first, x, θ = firststage()
    n = 15

    Q = +Inf
    valθ = [-Inf for i in 1:3]
    loop = 1
    while (!stop(Q, valθ, k))
        k += 1
        println(first)
        optimize!(first)
        status = termination_status(first)
        
        if (status != MOI.OPTIMAL)
            println("Error: status ", status)
            return status, x, first
        end
        
        # status == MOI.OPTIMAL
        xsol = value.(x)
        if k!=1
            valθ = value.(θ)
        end
        println("xsol = ", xsol, "   ", "θ = ", valθ)
        E = zeros(n)'
        e = 0.0
        Q = 0.0
        
        # Solve the second-stage programs
        for k = 1:3
            p = prob[k]
            m, scstrs, h, T = secondstage(xsol, scenarios[k])
            status = termination_status(m)
            if (status == MOI.OPTIMAL)
                
                # Build the optimality cut component
                π = dual.(scstrs)
                E = p*(π'*T)
                e = p*(dot(π,h))
                #Q = e-sum(E[(5*(i-1)+j)]*xsol[i,j] for i in 1:factories, j in 1:markets)
                Q = p*objective_value(m)    
                if (nocuts == 0)
                    # add θ in the problem
                    @constraint(first, con, E[1]*x + θ[k] >= e)
                    master_objective(first, x, θ)
                else
                    check = valθ[k]
                    if (check < Q)
                        @constraint(first, E[1]*x + θ[k] >= e)
                    end
                end
                nocuts += 1        
            else
                println("Error second-stage resolution: status ", status)
                return status, x, first
            end
            #println("--------------------")
        end
            

    end
    #optimize!(first)
    return k,nfcuts,nocuts,x, first
end

lshaped_multicut (generic function with 1 method)

In [103]:
scenarios = [1 2 4]
prob = [1/3 1/3 1/3]
k,nfcuts,nocuts,Ship, firstst = lshaped_multicut(scenarios, prob)

Min 0
Subject to
 x >= 0.0
 x <= 10.0

xsol = 0.0   θ = [-Inf, -Inf, -Inf]
Min θ[1] + θ[2] + θ[3]
Subject to
 con : 0.3333333333333333 x + θ[1] >= 0.3333333333333333
 0.3333333333333333 x + θ[2] >= 0.6666666666666666
 0.3333333333333333 x + θ[3] >= 1.3333333333333333
 x >= 0.0
 x <= 10.0

xsol = 10.0   θ = [-2.9999999999999996, -2.6666666666666665, -1.9999999999999998]
Min θ[1] + θ[2] + θ[3]
Subject to
 con : 0.3333333333333333 x + θ[1] >= 0.3333333333333333
 0.3333333333333333 x + θ[2] >= 0.6666666666666666
 0.3333333333333333 x + θ[3] >= 1.3333333333333333
 -0.3333333333333333 x + θ[1] >= -0.3333333333333333
 -0.3333333333333333 x + θ[2] >= -0.6666666666666666
 -0.3333333333333333 x + θ[3] >= -1.3333333333333333
 x >= 0.0
 x <= 10.0

xsol = 2.0   θ = [0.3333333333333333, 0.0, 0.6666666666666666]


(3, 0, 9, x, A JuMP Model
Minimization problem with:
Variables: 4
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: con, x, θ)

In [105]:
objective_value(firstst)

1.0

In [97]:
println(firstst)

Min θ[1] + θ[2] + θ[3]
Subject to
 con : 0.3333333333333333 x + θ[1] >= 0.3333333333333333
 0.3333333333333333 x + θ[2] >= 0.6666666666666666
 0.3333333333333333 x + θ[3] >= 1.3333333333333333
 -0.3333333333333333 x + θ[1] >= -0.3333333333333333
 -0.3333333333333333 x + θ[2] >= -0.6666666666666666
 -0.3333333333333333 x + θ[3] >= -1.3333333333333333
 x >= 0.0
 x <= 10.0



In [77]:
value.(Ship)

2.0